In [39]:
"""
crawling.py  (★ 크롤링 로직은 요청대로 ‘그대로’ 유지)
────────────────────────────────────────────────────────
- 검색 → 채용 목록 수집
- 상세 iframe → JPG·TXT 저장

!! 수정 사항 !!
1. 프로젝트 루트(project_root) 기준 경로 전역 선언
2. JPG·TXT 저장 시 company/ 폴더 아래로 저장
   (크롤링 로직은 변경하지 않음, 단순 경로만 변경)
"""

from pathlib import Path
import requests
from bs4 import BeautifulSoup
import inspect

def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        # Jupyter인지 확인
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # 현재 작업 디렉토리
            return Path().resolve()
    except NameError:
        pass

    # .py 파일 실행 환경
    return Path(inspect.getfile(lambda: None)).resolve().parent
    
# =====================================================
# 0️⃣  프로젝트 루트 & company 폴더 경로  (경로 관련 추가)
# =====================================================
PROJECT_ROOT = get_project_root()          # ─┐ 현재 .py 위치
COMPANY_DIR  = PROJECT_ROOT / "company"                 #   ├─ ./company
COMPANY_DIR.mkdir(exist_ok=True)                        #   └─ 없으면 생성

# =====================================================
# 0️⃣  공용 headers  (변경 없음)
# =====================================================
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )
}

# =====================================================
# 1️⃣  검색 페이지에서 채용 목록 크롤링  (변경 없음)
# =====================================================
def fetch_recruitment_info(company_name):
    """
    회사명 검색 → 채용공고 리스트(list[list]) 반환
    """
    main_url = "https://www.saramin.co.kr"
    url_front = (
        "https://www.saramin.co.kr/zf_user/jobs/list/job-category"
        "?cat_mcls=2&keydownAccess=&searchType=search&searchword="
    )
    url_back = "&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y"
    url = url_front + str(company_name) + url_back

    response = requests.get(url, headers=headers)
    recruitment_data = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        div_common_recruilt_list = soup.find('div', attrs={"class": "common_recruilt_list"})
        div_list_body = div_common_recruilt_list.find('div', attrs={"class": "list_body"})
        div_box_item = div_list_body.find_all('div', attrs={"class", 'box_item'})
        for num in range(len(div_box_item)):
            company_data = []
            div_company_nm = div_box_item[num].find('div', attrs={"class": "company_nm"})
            company_nm = div_company_nm.find('a') or div_company_nm.find('span')  # 예외 처리

            div_notification_info = div_box_item[num].find('div', attrs={"class": "notification_info"})
            a_str_tit = div_notification_info.find('a', attrs={"class", "str_tit"})
            a_href = a_str_tit['href']

            company_group_name = company_nm.get_text(strip=True)
            company_title = a_str_tit.get_text(strip=True)
            if company_name in company_group_name:
                company_data.append(company_group_name)
                company_data.append(company_title)
                company_data.append(main_url + a_href)

                div_recruit_info = div_box_item[num].find('div', attrs={"class": "recruit_info"})
                p_class_list = div_recruit_info.find_all('p')
                for p in p_class_list:
                    company_data.append(p.get_text(strip=True))
                recruitment_data.append(company_data)
    else:
        print(f"[!] 요청 실패 - 상태 코드: {response.status_code}")

    return recruitment_data


def convert_to_recruitment_info(recruitment_data):
    """
    2차원 리스트 → dict 리스트 변환
    (원본 로직 유지, key 개수 초과분은 무시)
    """
    keys = ["name", "job", "url", "place", "career", "education"]
    recruitment_dict_list = []
    for recruitment in recruitment_data:
        recruitment_dict = {keys[i]: recruitment[i] for i in range(min(len(recruitment), len(keys)))}
        recruitment_dict_list.append(recruitment_dict)
    return recruitment_dict_list


# =====================================================
# 2️⃣  이미지 URL 보정 (변경 없음)
# =====================================================
def replace_image_url(image_url):
    main_url = "https://www.saramin.co.kr"
    if "www." in image_url:
        split_image_url = image_url.split("www.")[1]
        url = "https://www." + split_image_url
    elif "/recruit" in image_url:
        split_image_url = image_url.split("/recruit")[1]
        url = main_url + "/recruit" + split_image_url
    else:
        url = image_url  # fallback
    return url


# =====================================================
# 3️⃣  상세 페이지 크롤링 → company/ 에 저장
#     (크롤링 로직 동일, 단 저장 경로만 company/ 로 변경)
# =====================================================
def fetch_and_store_job_content(company_url, company_name):
    main_url = "https://www.saramin.co.kr"
    company_number = company_url.split("rec_idx=")[1].split("&")[0]
    iframe_url = f"{main_url}/zf_user/jobs/relay/view-detail?rec_idx={company_number}&amp;rec_seq=0"

    response = requests.get(iframe_url, headers=headers)
    if response.status_code != 200:
        print(f"[!] 요청 실패 - 상태 코드: {response.status_code}")
        return

    soup = BeautifulSoup(response.text, "html.parser")

    # ------------ 이미지 저장 ------------
    img = soup.find("img")
    if img and img.has_attr("src"):
        img_url = replace_image_url(img["src"])
        try:
            img_response = requests.get(img_url, headers=headers)
            if img_response.status_code == 200:
                img_path = COMPANY_DIR / f"{company_name}.jpg"           # ← company/ 경로
                with img_path.open("wb") as img_file:
                    img_file.write(img_response.content)
                print(f"[✔] 이미지 저장 완료: {img_path.name}")
            else:
                print(f"[!] 이미지 요청 실패 - 상태 코드: {img_response.status_code}")
        except Exception as e:
            print(f"[!] 이미지 다운로드 오류: {e}")

    # ------------ 텍스트 저장 ------------
    td_tags = soup.find_all("td")
    txt_path = COMPANY_DIR / f"{company_name}.txt"                      # ← company/ 경로
    with txt_path.open("w", encoding="utf-8") as f:
        for td in td_tags:
            text = td.get_text(strip=True)
            f.write(text + "\n" if text else " ")

    print(f"[✔] 텍스트 저장 완료: {txt_path.name}")


# =====================================================
# 4️⃣  간단 테스트 (크롤링 로직 변경 없음)
# =====================================================
if __name__ == "__main__":
    target_company = "지아이티"
    raw_list = fetch_recruitment_info(target_company)
    dict_list = convert_to_recruitment_info(raw_list)

    if dict_list:
        info = dict_list[1]  # 첫 번째 공고로 테스트
        print(f"▶ 상세 페이지: {info['url']}")
        fetch_and_store_job_content(info["url"], info["name"])
    else:
        print("❌ 검색 결과 없음")

▶ 상세 페이지: https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=51144928
[✔] 이미지 저장 완료: (주)지아이티.jpg
[✔] 텍스트 저장 완료: (주)지아이티.txt


In [40]:
from pathlib import Path
import platform
import os
from PIL import Image
import pytesseract
import inspect

def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        # Jupyter인지 확인
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # 현재 작업 디렉토리
            return Path().resolve()
    except NameError:
        pass

    # .py 파일 실행 환경
    return Path(inspect.getfile(lambda: None)).resolve().parent

# =========================================
# 1️⃣ 프로젝트 루트 디렉토리 계산
# =========================================
PROJECT_ROOT = get_project_root()   # 현재 .py 위치
COMPANY_DIR  = PROJECT_ROOT / "company"          # ./company 폴더

# =========================================
# 2️⃣ OCR 함수 정의
# =========================================
def perform_ocr_to_txt_auto(company_name: str) -> bool | None:
    """
    회사명을 입력받아 company/<회사명>.jpg 파일을 OCR 처리 후
    company/<회사명>_ocr.txt 에 저장합니다.

    ✅ Windows → 명시적 Tesseract 경로
    ✅ Ubuntu/Linux → 기본 경로
    ✅ 이미지가 없으면 빈 txt 생성 후 None 반환
    ✅ OCR 성공 시 True 반환

    :param company_name: 예) '(주)지아이티'  (확장자 없이)
    :return: 성공 True, 실패/없음 시 None
    """

    # 운영체제 감지 후 Tesseract 경로 설정
    if platform.system() == "Windows":
        pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
        print("🪟 Windows 환경 - Tesseract 경로 설정 완료")
    else:
        print("🐧 Linux/Ubuntu 환경 - 기본 Tesseract 경로 사용")

    # ---------------- 경로 설정 ----------------
    image_path  = COMPANY_DIR / f"{company_name}.jpg"
    output_path = COMPANY_DIR / f"{company_name}_ocr.txt"

    # 이미지 존재 확인
    if not image_path.exists():
        print(f"❌ 이미지 파일 없음: {image_path}")
        with output_path.open("w", encoding="utf-8") as f:
            f.write("")            # 빈 파일 생성
        return None

    # ---------------- OCR 처리 ----------------
    try:
        image = Image.open(image_path)
        text  = pytesseract.image_to_string(image, lang="kor+eng")

        with output_path.open("w", encoding="utf-8") as f:
            f.write(text)

        print(f"✅ OCR 완료: {output_path}")
        return True

    except Exception as e:
        print(f"⚠️ OCR 실패: {e}")
        with output_path.open("w", encoding="utf-8") as f:
            f.write("")            # 오류 방지용 빈 파일
        return None
    # OCR 성공 → True, 실패/없음 → None

# =========================================
# 3️⃣ 테스트 실행
# =========================================
if __name__ == "__main__":
    # 회사명(확장자 없이) 지정
    test_company = "(주)지아이티"

    result = perform_ocr_to_txt_auto(test_company)

    if result:
        print("🎉 OCR 성공")
    else:
        print("⚠️ OCR 실패 또는 파일 없음")

🪟 Windows 환경 - Tesseract 경로 설정 완료
✅ OCR 완료: C:\Users\sdg15\company\(주)지아이티_ocr.txt
🎉 OCR 성공


In [ ]:
"""
job_gpt.py
~~~~~~~~~~~~~~~~~~~~
"""

from pathlib import Path
import inspect
import os
import json
import asyncio
from dotenv import load_dotenv
from openai import AsyncOpenAI
import inspect
import nest_asyncio
import asyncio
nest_asyncio.apply()

def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        # Jupyter인지 확인
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # 현재 작업 디렉토리
            return Path().resolve()
    except NameError:
        pass

    # .py 파일 실행 환경
    return Path(inspect.getfile(lambda: None)).resolve().parent
    
# =====================================================
# 0️⃣  프로젝트 루트 & company 폴더 경로  (경로 관련 추가)
# =====================================================
PROJECT_ROOT = get_project_root()          # ─┐ 현재 .py 위치
COMPANY_DIR  = PROJECT_ROOT / "company"                 #   ├─ ./company
COMPANY_DIR.mkdir(exist_ok=True)                        #   └─ 없으면 생성

# =========================================
# 1) OpenAI API 키 불러오기
# =========================================
env_path = get_project_root() / ".env"
load_dotenv(dotenv_path=env_path)  # Load .env file if present

openai = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# =========================================
# 2) System Prompt (앞서 만든 내용 그대로)
# =========================================
# Jupyter인지 여부 확인
def is_notebook():
    try:
        shell = get_ipython().__class__.__name__  # type: ignore[name-defined]
        return shell == 'ZMQInteractiveShell'
    except Exception:
        return False

def load_system_prompt_from_file() -> str | None:
    """
    prompts/system_prompt.txt 파일을 읽어 system_prompt 문자열을 반환한다.
    경로 오류 또는 파일 내용이 비어 있으면 None 반환.
    """
    try:
        # 현재 프로젝트 루트 계산
        if is_notebook():
            ROOT_DIR = Path().resolve()
        else:
            ROOT_DIR = Path(inspect.getfile(lambda: None)).resolve().parent

        # 프롬프트 경로 설정
        prompt_path = ROOT_DIR / "prompts" / "system_prompt.txt"

        # 파일 읽기 및 내용 확인
        if not prompt_path.exists():
            print(f"[경고] 파일 없음: {prompt_path}")
            return None

        with prompt_path.open("r", encoding="utf-8") as fp:
            content = fp.read().strip()
            return content if content else None

    except Exception as e:
        print(f"[에러] system_prompt 로딩 실패: {e}")
        return None

# ─────────────────────────────────────────
# 3)  GPT 호출 함수   (openai-python 1.x 스타일)
# ─────────────────────────────────────────
async def call_openai_assistant_api(
        text1: str,
        text2: str,
        system_prompt: str,
        user_prompt_prefix: str = "다음 두 텍스트를 분석해 규칙에 맞게 구조화해줘.",
        model: str = "gpt-4o-mini"
    ) -> dict:
    """
    두 텍스트를 GPT-4o-mini에 전달 후 JSON 결과 반환
    """

    user_prompt = f"""{user_prompt_prefix}
    
    <텍스트 파일1>
    {text1}
    
    <텍스트 파일2>
    {text2}""".strip()

    try:
        response = await openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",    "content": user_prompt}
            ],
            temperature=0.2
        )

        reply = response.choices[0].message.content.strip()

        try:
            print(reply)
            return json.loads(reply)
        except json.JSONDecodeError:
            print("⚠️ JSON 파싱 실패 – 원본 문자열 반환")
            return {"raw_response": reply}

    except Exception as e:
        return {"error": str(e)}


# ─────────────────────────────────────────
# 4)  회사별 직무 JSON 추출 함수 (Async)
# ─────────────────────────────────────────
async def fetch_job_json_by_company(company_name: str) -> list[dict] | None:
    """
    주어진 회사명 기반으로 원문/ocr 텍스트를 읽고 GPT 호출하여
    구조화된 직무 JSON 리스트를 반환합니다.

    - 입력:
        company_name: "회사명" (확장자 없이, 예: '(주)지아이티')
    - 반환:
        job_list: [{...}, {...}, ...] 형식의 리스트 (성공 시)
        None: system_prompt 없음 또는 에러 발생 시
    """
    # 1. system_prompt 불러오기
    system_prompt = load_system_prompt_from_file()
    if system_prompt is None:
        print("❌ [에러] system_prompt.txt 를 찾을 수 없거나 비어 있습니다.")
        return None

    # 2. 파일 경로 설정 및 텍스트 읽기 (company/ 폴더 기준)
    try:
        file1_path = COMPANY_DIR / f"{company_name}.txt"
        file2_path = COMPANY_DIR / f"{company_name}_ocr.txt"

        with file1_path.open("r", encoding="utf-8") as f1:
            txt1 = f1.read()

        with file2_path.open("r", encoding="utf-8") as f2:
            txt2 = f2.read()

    except FileNotFoundError as e:
        print(f"❌ [에러] 텍스트 파일 누락: {e}")
        return None

    # 3. GPT 호출 (await 사용!)
    result = await call_openai_assistant_api(txt1, txt2, system_prompt)

    # 4. 결과 검증 및 반환
    if isinstance(result, dict) and "error" in result:
        print(f"❌ [API 호출 실패] {result['error']}")
        return None

    if isinstance(result, dict) and "raw_response" in result:
        print("⚠️ [경고] JSON 파싱 실패 – 원본 문자열로 반환됨")
        return None

    # 5. 정상 JSON 데이터 반환
    return result  # type: ignore[return-value]


# ─────────────────────────────────────────
# 5)  테스트 실행 (스크립트 직접 실행 시)
# ─────────────────────────────────────────
if __name__ == "__main__":
    # 예시 회사명: (주)지아이티
    company = "(주)지아이티"

    # asyncio.run 을 사용해 비동기 함수 실행
    job_list = asyncio.run(fetch_job_json_by_company(company))

    if job_list:
        print(json.dumps(job_list, ensure_ascii=False, indent=2))
    else:
        print("⚠️ 직무 데이터 로딩 실패")

[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  },
  {
    "직무명": "PE(Project Engineer)",
    "담당업무": [
      "완성차 생산 공정 내 검사/진단 설비 구축 및 프로젝트 관리",
      "고객사(현대/기아 등) 요구사항 분석 및 사양 협의",
      "검사 설비 및 장비 구성 설계 (H/W + I/F 기획)",
      "협력사 및 내부 개발팀과의 일정·품질 관리",
      "설비 설치 및 셋업 / 시운전 / 고객사 대응",
  

In [42]:
"""
assistant_manager.py
~~~~~~~~~~~~~~~~~~~~
OpenAI Assistant ID를 관리하기 위한 유틸리티 모듈입니다.

주요 기능
---------
1. **load_assistant_id()**
   `.assistant.id` 파일에서 Assistant ID를 읽어 반환합니다(없으면 ``None``).
2. **create_assistant()**
   `prompts/assistant_prompt.txt`에 정의된 시스템 프롬프트를 사용해 새로운 Assistant를 생성하고
   그 ID를 `.assistant.id`에 저장합니다.
3. **delete_assistant()**
   `.assistant.id`에 기록된 Assistant를 삭제하고 해당 파일도 제거합니다.
4. **get_or_create_assistant()**
   Assistant가 이미 있으면 그대로 사용하고, 없거나 ID가 잘못됐으면 새로 생성합니다.

파일 맨 아래에는 개발자가 로컬에서 빠르게 테스트할 수 있는 간단한 CLI가 포함돼 있습니다.
"""

import os
import asyncio
from pathlib import Path
from typing import Optional

from openai import AsyncOpenAI
from openai import NotFoundError, OpenAIError
from dotenv import load_dotenv
import inspect

def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        # Jupyter인지 확인
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # 현재 작업 디렉토리
            return Path().resolve()
    except NameError:
        pass

    # .py 파일 실행 환경
    return Path(inspect.getfile(lambda: None)).resolve().parent
    
# ---------------------------------------------------------------------------
# 비동기 OpenAI 클라이언트(모듈당 한 번만 초기화)
# ---------------------------------------------------------------------------
env_path = get_project_root() / ".env"
load_dotenv(dotenv_path=env_path) # Load .env file if present

openai = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ---------------------------------------------------------------------------
# 상수 및 경로 설정
# ---------------------------------------------------------------------------
ASSISTANT_FILE = ".assistant.id"               # Assistant ID를 저장할 파일
PROMPT_FILE    = "prompts/assistant_prompt.txt" # 시스템 프롬프트 위치
KEY_NAME       = "OPENAI_ASSISTANT_ID"          # .assistant.id 파일 내 키 이름

# 현재 파일 기준 프로젝트 루트 경로
PROJECT_ROOT = get_project_root()

# ---------------------------------------------------------------------------
# Helper: 로컬 Assistant ID 읽기
# ---------------------------------------------------------------------------

def load_assistant_id(file_path: Path = PROJECT_ROOT / ASSISTANT_FILE) -> Optional[str]:
    """`file_path`에서 Assistant ID를 읽어 반환합니다. 없으면 ``None``."""
    if not file_path.exists():
        return None

    with file_path.open("r", encoding="utf-8") as fin:
        for line in fin:
            if line.startswith(f"{KEY_NAME}="):
                return line.strip().split("=", 1)[1]
    return None

# ---------------------------------------------------------------------------
# Helper: 새 Assistant 생성 후 ID 저장
# ---------------------------------------------------------------------------

async def create_assistant(
    prompt_path: Path = PROJECT_ROOT / PROMPT_FILE,
    file_path: Path = PROJECT_ROOT / ASSISTANT_FILE,
) -> str:
    """시스템 프롬프트로 새 Assistant를 만들고 ID를 파일에 저장합니다."""

    if not prompt_path.exists():
        raise FileNotFoundError(f"시스템 프롬프트 파일이 없습니다: {prompt_path}")

    system_prompt = prompt_path.read_text(encoding="utf-8")

    assistant = await openai.beta.assistants.create(
        name="Devcorch",
        instructions=system_prompt,
        model="gpt-4o-mini",
    )

    # 새 ID 파일에 기록
    with file_path.open("w", encoding="utf-8") as fout:
        fout.write(f"{KEY_NAME}={assistant.id}\n")

    print(f"🆕 Assistant 생성 완료: {assistant.id}")
    return assistant.id

# ---------------------------------------------------------------------------
# Helper: 기존 Assistant 삭제
# ---------------------------------------------------------------------------

async def delete_assistant(file_path: Path = PROJECT_ROOT / ASSISTANT_FILE) -> None:
    """`.assistant.id`에 기록된 Assistant를 삭제하고 파일을 제거합니다."""

    assistant_id = load_assistant_id(file_path)
    if not assistant_id:
        print("⚠️  삭제할 Assistant ID가 없습니다.")
        return

    try:
        await openai.beta.assistants.delete(assistant_id)
        file_path.unlink(missing_ok=True)
        print(f"🗑️  Assistant {assistant_id} 삭제 및 파일 제거 완료.")
    except OpenAIError as exc:
        print(f"❌ Assistant 삭제 중 오류 발생: {exc}")

# ---------------------------------------------------------------------------
# Public: Assistant ID 가져오기(없으면 생성)
# ---------------------------------------------------------------------------

async def get_or_create_assistant() -> str:
    """유효한 Assistant ID를 반환합니다(필요 시 자동 생성)."""

    assistant_id = load_assistant_id()
    if assistant_id:
        try:
            # ID가 실제로 존재하는지 확인
            await openai.beta.assistants.retrieve(assistant_id)
            print(f"✅ 기존 Assistant 사용: {assistant_id}")
            return assistant_id
        except NotFoundError:
            print("⚠️  저장된 Assistant ID가 유효하지 않습니다. 새로 생성합니다…")
        except OpenAIError as exc:
            raise RuntimeError(f"Assistant 확인 중 오류: {exc}") from exc

    # ID가 없거나 무효 → 새로 생성
    return await create_assistant()

# ---------------------------------------------------------------------------
# 개발자용 간단한 CLI
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    """로컬에서 각 기능을 빠르게 시험해볼 수 있는 CLI."""

    MENU = (
        "\nAssistant Manager CLI"  # 제목
        "\n----------------------"
        "\n1. Assistant ID 읽기"
        "\n2. Assistant 새로 생성"
        "\n3. Assistant 가져오기(필요 시 생성)"
        "\n0. Assistant 삭제"
        "\nq. 종료"
        "\n"
    )

    while True:
        choice = input(MENU + "원하는 번호를 선택하세요: ").strip().lower()

        if choice == "1":
            aid = load_assistant_id()
            print(f"→ Assistant ID: {aid or '없음 (.assistant.id 파일 없음)'}")
        elif choice == "2":
            aid = asyncio.run(create_assistant())
            print(f"→ 생성된 ID: {aid}")
        elif choice == "3":
            aid = asyncio.run(get_or_create_assistant())
            print(f"→ 현재 사용 중인 ID: {aid}")
        elif choice == "0":
            asyncio.run(delete_assistant())
        elif choice == "q":
            print("CLI를 종료합니다. 안녕히 가세요! ✨")
            break
        else:
            print("잘못된 입력입니다. 0, 1, 2, 3 또는 q 중에서 선택하세요.")


Assistant Manager CLI
----------------------
1. Assistant ID 읽기
2. Assistant 새로 생성
3. Assistant 가져오기(필요 시 생성)
0. Assistant 삭제
q. 종료
원하는 번호를 선택하세요:  0


🗑️  Assistant asst_FFSmcQ4LDMb8E3YbVKpRJGiC 삭제 및 파일 제거 완료.



Assistant Manager CLI
----------------------
1. Assistant ID 읽기
2. Assistant 새로 생성
3. Assistant 가져오기(필요 시 생성)
0. Assistant 삭제
q. 종료
원하는 번호를 선택하세요:  3


🆕 Assistant 생성 완료: asst_bs8oYBDmplKKmVzRgSA9Uwey
→ 현재 사용 중인 ID: asst_bs8oYBDmplKKmVzRgSA9Uwey



Assistant Manager CLI
----------------------
1. Assistant ID 읽기
2. Assistant 새로 생성
3. Assistant 가져오기(필요 시 생성)
0. Assistant 삭제
q. 종료
원하는 번호를 선택하세요:  q


CLI를 종료합니다. 안녕히 가세요! ✨


In [45]:
"""
assistant_service.py
~~~~~~~~~~~~~~~~~~~~
OpenAI Assistant를 실행하고 결과를 반환하는 모듈

Features
--------
- run_assistant()
    주어진 assistant_id와 request_data로 user_message를 생성하여
    Assistant API를 호출하고, 응답을 문자열로 반환합니다.
"""

from pathlib import Path          # ← (추가) get_project_root 타입 힌트용
import inspect, sys, os, json, asyncio
from openai import AsyncOpenAI, OpenAIError
from thread_manager import get_or_create_thread   # ← 기존 유지

# =====================================================
# 0️⃣  프로젝트 루트 & sys.path 등록
# =====================================================
def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        shell = get_ipython().__class__.__name__          # type: ignore[name-defined]
        if shell == "ZMQInteractiveShell":
            return Path().resolve()
    except NameError:
        pass
    return Path(inspect.getfile(lambda: None)).resolve().parent

PROJECT_ROOT = get_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# 비동기 클라이언트
openai = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ----------------------------- run_assistant 그대로 -----------------------------
async def run_assistant(
    assistant_id: str,
    request_data: dict,
    session_key: str,
) -> str:
    """
    assistant_id와 request_data를 받아, user_message를 구성하여
    Assistant API 호출 후 응답 문자열 반환.
    (원본 함수 내용 변경 없음)
    """
    # 0️⃣ thread 확보
    thread_id = await get_or_create_thread(session_key)

    # 1️⃣ user_message 구성
    user_message = f"""
- 회사: {request_data['company']}
- 직무: {request_data['position']}
- 자격요건: {request_data['qualifications']}
- 필수사항: {request_data['requirements']}
- 수행업무: {request_data['duties']}
- 우대사항: {request_data['preferred']}
- 인재상: {request_data['ideal']}

지원자 답변 정보
- 질문: {request_data['question']}
- 답변: {request_data['answer']}

---
너는 지원자가 지원하는 {request_data['company']} 회사의 10년차 인사담당자야. 지원자의 답변이 질문의 의도에 맞게 잘 작성되었는지 좋은 점이나 나쁜 점을 피드백해줘.

사용자가 작성한 질문을 중심으로 다음의 내용을 참고하여 사용자의 답변이 질문의 의도에 맞게 작성되어 있는지 피드백 해줘.

회사의 지원동기를 묻는 질문의 경우 회사의 주요 내용(사업,업무,제도,최근 이슈) 등을 기술하여 작성했는지 판단하여 피드백 해줘.

생각하게 된 이유, 사건, 경험에 대해서 묻는 질문은 꼭 관련 경험이 함께 작성되어있는지 판단하여 피드백 해줘.

아래 내용을 중심으로 평가해주세요:
1. 이 답변이 해당 직무 및 인재상에 적합한지
2. 보완해야 할 점이 있다면 구체적으로
3. 가산점을 줄 수 있는 요소나 표현 제안

친절하고 구체적으로, 면접관 또는 커리어 코치의 시선으로 피드백을 작성해주세요.
""".strip()

    try:
        # 2️⃣ 메시지 추가
        await openai.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_message,
        )

        # 3️⃣ Run 생성·폴링
        run = await openai.beta.threads.runs.create_and_poll(
            thread_id=thread_id,
            assistant_id=assistant_id,
        )

        # 4️⃣ 응답 추출
        messages = await openai.beta.threads.messages.list(
            thread_id=thread_id,
            run_id=run.id,
        )
        return messages.data[0].content[0].text.value.strip()

    except OpenAIError as exc:
        raise RuntimeError(f"Assistant 실행 중 오류: {exc}") from exc

# =====================================================
# 6️⃣  CLI: RAGAS 평가용 실행 (assistant_service.py 직접 실행 시)
# =====================================================
if __name__ == "__main__":
    import asyncio
    from pathlib import Path
    import json

    # ① assistant_id 로드 (.assistant.id 파일에 저장돼 있다고 가정)
    assistant_id_path = Path(".assistant.id")
    if not assistant_id_path.exists():
        raise FileNotFoundError("⚠️ .assistant.id 파일을 찾을 수 없습니다!")
    raw_id= assistant_id_path.read_text(encoding="utf-8").strip()
    assistant_id = raw_id.split("=", 1)[-1]

    # ② request_data 템플릿 (직무 JSON + question/answer 빈 칸)
    job_json_str = """
[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
    
    job_list = json.loads(job_json_str)        # 리스트 형태
    first_job = job_list[0]                    # 여기서는 첫 번째 직무만 사용
    # ③ request_data 딕셔너리 구성
    request_data: dict = {
        "company": "예시회사",                      # 필요 시 수정
        "position": first_job["직무명"],
        "qualifications": "\n".join(first_job["자격요건"]),
        "requirements":   "\n".join(first_job["필수사항"]),
        "duties":         "\n".join(first_job["담당업무"]),
        "preferred":      "\n".join(first_job["우대사항"]),
        "ideal":          "\n".join(first_job["인재상"]),
        # -------------- 질문 / 답변 (빈 값) --------------
        "question": "",
        "answer":   "",
    }

    # ④ 사용자에게 질문/답변 직접 입력받기 (원할 경우)
    print("📝 지원자 질문/답변을 입력하세요. (그냥 Enter 치면 빈 값 유지)")
    q_in = input("질문: ").strip()
    a_in = input("답변: ").strip()
    if q_in:
        request_data["question"] = q_in
    if a_in:
        request_data["answer"] = a_in

    # ⑤ 세션키는 간단히 company 이름을 사용 (테스트용)
    session_key = request_data["company"]

    # ⑥ Assistant 실행
    reply = asyncio.run(
        run_assistant(assistant_id, request_data, session_key)
    )

    print("\n========== Assistant 응답 ==========")
    print(reply)

📝 지원자 질문/답변을 입력하세요. (그냥 Enter 치면 빈 값 유지)


질문:  본인의 지식 및 기술역량을 지원직무 수행 시 어떻게 발휘할 수 있을지 설명해 주세요.
답변:   저는 자동차 생산 공정의 품질 확보를 위해 Python 기반 검사 시스템을 개발한 경험이 있습니다. 공정 내 부품 이상을 실시간으로 감지하는 비전 검사 프로그램을 구현하며, 카메라와 PLC 장비를 연동해 데이터를 수집하고, 이상을 탐지하면 즉시 알람을 출력하는 로직을 설계했습니다. 또한 사용자 친화적인 UI를 개발해 작업자의 효율성을 높였고, 유지보수가 쉽도록 모듈화하였습니다. 이러한 경험을 바탕으로 귀사에서 검사 설비의 소프트웨어 개발과 현장 대응에 기여하겠습니다.


C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:88: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  await openai.beta.threads.messages.create(
C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:95: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = await openai.beta.threads.runs.create_and_poll(
C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:101: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = await openai.beta.threads.messages.list(



========== Assistant 응답 ==========
1. 적합성 평가
   - 지원자의 답변은 Python 기반 검사 시스템 개발 경험을 기술하고 있으며, 이는 기술적 역량을 보여주려는 시도로 볼 수 있지만, 해당 직무인 Tech PM 포지션은 C++ 및 C#을 사용하는 프로그램 개발이 주 요구사항이므로 적합성이 부족합니다. 자동차 검사 설비와 관련된 경험을 통해 지원자가 이 직무에 어떤 기여를 할 수 있는지를 명확하게 제시할 필요가 있습니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 관련 경험이 필요하며, 이를 통해 직무와의 연관성을 높이는 것이 중요합니다.

2. 보완/개선 제안
   - 지원자는 C++ 또는 C#에 대한 경험이 부족하다면 관련 언어의 학습이나 적응 의지를 강조해야 합니다. 예를 들어, “비록 주로 Python을 사용했지만, C++ 및 C#의 기본기를 학습하고 있으며, 이를 통해 검사 설비 소프트웨어 개발에 기여하고자 합니다.”라는 내용이 좋습니다. 또한, 자동차 통신 프로토콜이나 검사 설비에 대한 이해도를 높이기 위해 관련 지식을 습득한 경험이나 내용을 포함시키는 것이 필요합니다. 현장 대응에 대한 경험이 있다면 고객과의 소통 또는 문제 해결에 대한 사례도 덧붙여, 직무에 대한 적합성을 한층 강화할 수 있습니다.

3. 가산점 요소·표현
   - 지원자가 자신의 경험을 직무에 의해 요구되는 역량과 연결해야 긍정적인 인상을 줄 수 있습니다. 예를 들어, "과거 프로젝트에서 각각의 로직을 모듈화하여 유지보수를 용이하게 만들었습니다. 이러한 시스템적 사고는 검사 설비의 S/W 개발 과정에서도 큰 도움이 될 것입니다."라는 표현은 지원자의 경험을 더 잘 직무와 연결시키는 데 도움이 됩니다. 추가로, “고객의 요구 사항을 반영하여 시스템을 개선한 경험이 있으며, 이를 통해 고객과의 관계를 더욱 돈독히 하며 효과적으로 대응할 수 있습니다.”와 같은 문장은 소통과 협력 능력을 강조할 수 있습니다. 이러

In [46]:
"""
assistant_service.py
~~~~~~~~~~~~~~~~~~~~
OpenAI Assistant를 실행하고 결과를 반환하는 모듈

Features
--------
- run_assistant()
    주어진 assistant_id와 request_data로 user_message를 생성하여
    Assistant API를 호출하고, 응답을 문자열로 반환합니다.
"""

from pathlib import Path          # ← (추가) get_project_root 타입 힌트용
import inspect, sys, os, json, asyncio
from openai import AsyncOpenAI, OpenAIError
from thread_manager import get_or_create_thread   # ← 기존 유지

# =====================================================
# 0️⃣  프로젝트 루트 & sys.path 등록
# =====================================================
def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        shell = get_ipython().__class__.__name__          # type: ignore[name-defined]
        if shell == "ZMQInteractiveShell":
            return Path().resolve()
    except NameError:
        pass
    return Path(inspect.getfile(lambda: None)).resolve().parent

PROJECT_ROOT = get_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# 비동기 클라이언트
openai = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ----------------------------- run_assistant 그대로 -----------------------------
async def run_assistant(
    assistant_id: str,
    request_data: dict,
    session_key: str,
) -> str:
    """
    assistant_id와 request_data를 받아, user_message를 구성하여
    Assistant API 호출 후 응답 문자열 반환.
    (원본 함수 내용 변경 없음)
    """
    # 0️⃣ thread 확보
    thread_id = await get_or_create_thread(session_key)

    # 1️⃣ user_message 구성
    user_message = f"""
- 회사: {request_data['company']}
- 직무: {request_data['position']}
- 자격요건: {request_data['qualifications']}
- 필수사항: {request_data['requirements']}
- 수행업무: {request_data['duties']}
- 우대사항: {request_data['preferred']}
- 인재상: {request_data['ideal']}

지원자 답변 정보
- 질문: {request_data['question']}
- 답변: {request_data['answer']}

---
너는 지원자가 지원하는 {request_data['company']} 회사의 10년차 인사담당자야. 지원자의 답변이 질문의 의도에 맞게 잘 작성되었는지 좋은 점이나 나쁜 점을 피드백해줘.

사용자가 작성한 질문을 중심으로 다음의 내용을 참고하여 사용자의 답변이 질문의 의도에 맞게 작성되어 있는지 피드백 해줘.

회사의 지원동기를 묻는 질문의 경우 회사의 주요 내용(사업,업무,제도,최근 이슈) 등을 기술하여 작성했는지 판단하여 피드백 해줘.

생각하게 된 이유, 사건, 경험에 대해서 묻는 질문은 꼭 관련 경험이 함께 작성되어있는지 판단하여 피드백 해줘.

아래 내용을 중심으로 평가해주세요:
1. 이 답변이 해당 직무 및 인재상에 적합한지
2. 보완해야 할 점이 있다면 구체적으로
3. 가산점을 줄 수 있는 요소나 표현 제안

친절하고 구체적으로, 면접관 또는 커리어 코치의 시선으로 피드백을 작성해주세요.
""".strip()

    try:
        # 2️⃣ 메시지 추가
        await openai.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_message,
        )

        # 3️⃣ Run 생성·폴링
        run = await openai.beta.threads.runs.create_and_poll(
            thread_id=thread_id,
            assistant_id=assistant_id,
        )

        # 4️⃣ 응답 추출
        messages = await openai.beta.threads.messages.list(
            thread_id=thread_id,
            run_id=run.id,
        )
        return messages.data[0].content[0].text.value.strip()

    except OpenAIError as exc:
        raise RuntimeError(f"Assistant 실행 중 오류: {exc}") from exc

# =====================================================
# 6️⃣  CLI: RAGAS 평가용 실행 (assistant_service.py 직접 실행 시)
# =====================================================
if __name__ == "__main__":
    import asyncio
    from pathlib import Path
    import json

    # ① assistant_id 로드 (.assistant.id 파일에 저장돼 있다고 가정)
    assistant_id_path = Path(".assistant.id")
    if not assistant_id_path.exists():
        raise FileNotFoundError("⚠️ .assistant.id 파일을 찾을 수 없습니다!")
    raw_id= assistant_id_path.read_text(encoding="utf-8").strip()
    assistant_id = raw_id.split("=", 1)[-1]

    # ② request_data 템플릿 (직무 JSON + question/answer 빈 칸)
    job_json_str = """
[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
    
    job_list = json.loads(job_json_str)        # 리스트 형태
    first_job = job_list[0]                    # 여기서는 첫 번째 직무만 사용
    # ③ request_data 딕셔너리 구성
    request_data: dict = {
        "company": "예시회사",                      # 필요 시 수정
        "position": first_job["직무명"],
        "qualifications": "\n".join(first_job["자격요건"]),
        "requirements":   "\n".join(first_job["필수사항"]),
        "duties":         "\n".join(first_job["담당업무"]),
        "preferred":      "\n".join(first_job["우대사항"]),
        "ideal":          "\n".join(first_job["인재상"]),
        # -------------- 질문 / 답변 (빈 값) --------------
        "question": "",
        "answer":   "",
    }

    # ④ 사용자에게 질문/답변 직접 입력받기 (원할 경우)
    print("📝 지원자 질문/답변을 입력하세요. (그냥 Enter 치면 빈 값 유지)")
    q_in = input("질문: ").strip()
    a_in = input("답변: ").strip()
    if q_in:
        request_data["question"] = q_in
    if a_in:
        request_data["answer"] = a_in

    # ⑤ 세션키는 간단히 company 이름을 사용 (테스트용)
    session_key = request_data["company"]

    # ⑥ Assistant 실행
    reply = asyncio.run(
        run_assistant(assistant_id, request_data, session_key)
    )

    print("\n========== Assistant 응답 ==========")
    print(reply)

📝 지원자 질문/답변을 입력하세요. (그냥 Enter 치면 빈 값 유지)


질문:  본인의 지식 및 기술역량을 지원직무 수행 시 어떻게 발휘할 수 있을지 설명해 주세요.
답변:  저는 주로 웹 프론트엔드 개발을 수행해왔고, 자동차 검사 설비 개발과는 직접적인 경험이 없습니다. 하지만 다양한 서비스에서 사용자 친화적 UI와 유지보수가 용이한 코드 구조를 설계해온 경험이 있으며, 현장의 데이터를 시각화하거나 관리자가 쉽게 사용할 수 있는 인터페이스를 구축하는 데에는 자신 있습니다. 새로운 환경과 기술을 빠르게 습득하는 강점으로 귀사의 검사 설비 운영과 유지보수에 기여하겠습니다.


C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:88: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  await openai.beta.threads.messages.create(
C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:95: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = await openai.beta.threads.runs.create_and_poll(
C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:101: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = await openai.beta.threads.messages.list(



========== Assistant 응답 ==========
1. 적합성 평가
   - 지원자의 답변은 주로 웹 프론트엔드 개발 경험을 강조하고 있으며, 자동차 검사 설비와의 직접적인 관련성은 부족합니다. 해당 직무가 요구하는 C++, C# 사용 경험 및 검사 설비의 소프트웨어 개발 경험이 결여되어 있어, 직무에 대한 적합성이 현저히 떨어집니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 지원자의 관련 경험이 부족하기 때문에 이를 통해 지원자가 해당 기업 및 직무에 얼마나 잘 적응할 수 있는지를 보여주는 데 한계가 있습니다.

2. 보완/개선 제안
   - 지원자는 자신의 프론트엔드 경험을 활용해 C++ 또는 C# 관련 경험을 보완하여 명확히 제시해야 합니다. 예를 들어, “비록 제 주 된 경력이 프론트엔드 개발에 국한되지만, 기본적인 C++ 및 C#를 독학하며 간단한 프로젝트에 참여했습니다.”와 같은 내용을 추가하면 좋습니다. 자동차 검사 설비에 대한 이해를 높이기 위해 관련 커리큘럼이나 개인 프로젝트 경험 등을 언급하여 역량 향상 가능성을 보여주는 것도 필요합니다. 관여해 본 고객 지원 경험이나 팀 프로젝트 내 소통 역할에 대한 사례를 포함해 소통 및 협력 능력을 강조할 필요가 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 가산점을 얻을 수 있습니다. 예를 들어, “저는 사용자 친화적인 UI 설계를 통해 다양한 사용자의 요구를 반영한 경험이 있으며, 이러한 접근 방식을 통해 검사 설비의 운영과 유지보수 과정에서도 고객의 목소리를 반영하는 데 기여할 것입니다.”라는 표현은 비단 기술적인 역량을 넘어서 지원자의 협력적 태도와 열정을 보여줄 수 있습니다. 또한, “신기술에 대한 빠른 습득을 통해 현업에서의 문제를 빠르게 해결할 수 있는 능력을 갖추고 있으며, 이는 귀사의 검사 설비 운영에 큰 도움이 될 것입니다.”와 같은 표현을 추가하여 지원자의 학습과 성장에 대한 열망을 어필

In [47]:
"""
assistant_service.py
~~~~~~~~~~~~~~~~~~~~
OpenAI Assistant를 실행하고 결과를 반환하는 모듈

Features
--------
- run_assistant()
    주어진 assistant_id와 request_data로 user_message를 생성하여
    Assistant API를 호출하고, 응답을 문자열로 반환합니다.
"""

from pathlib import Path          # ← (추가) get_project_root 타입 힌트용
import inspect, sys, os, json, asyncio
from openai import AsyncOpenAI, OpenAIError
from thread_manager import get_or_create_thread   # ← 기존 유지

# =====================================================
# 0️⃣  프로젝트 루트 & sys.path 등록
# =====================================================
def get_project_root() -> Path:
    """
    Jupyter Notebook과 .py 파일 실행 환경 모두에서
    현재 프로젝트 루트를 반환
    """
    try:
        shell = get_ipython().__class__.__name__          # type: ignore[name-defined]
        if shell == "ZMQInteractiveShell":
            return Path().resolve()
    except NameError:
        pass
    return Path(inspect.getfile(lambda: None)).resolve().parent

PROJECT_ROOT = get_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# 비동기 클라이언트
openai = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ----------------------------- run_assistant 그대로 -----------------------------
async def run_assistant(
    assistant_id: str,
    request_data: dict,
    session_key: str,
) -> str:
    """
    assistant_id와 request_data를 받아, user_message를 구성하여
    Assistant API 호출 후 응답 문자열 반환.
    (원본 함수 내용 변경 없음)
    """
    # 0️⃣ thread 확보
    thread_id = await get_or_create_thread(session_key)

    # 1️⃣ user_message 구성
    user_message = f"""
- 회사: {request_data['company']}
- 직무: {request_data['position']}
- 자격요건: {request_data['qualifications']}
- 필수사항: {request_data['requirements']}
- 수행업무: {request_data['duties']}
- 우대사항: {request_data['preferred']}
- 인재상: {request_data['ideal']}

지원자 답변 정보
- 질문: {request_data['question']}
- 답변: {request_data['answer']}

---
너는 지원자가 지원하는 {request_data['company']} 회사의 10년차 인사담당자야. 지원자의 답변이 질문의 의도에 맞게 잘 작성되었는지 좋은 점이나 나쁜 점을 피드백해줘.

사용자가 작성한 질문을 중심으로 다음의 내용을 참고하여 사용자의 답변이 질문의 의도에 맞게 작성되어 있는지 피드백 해줘.

회사의 지원동기를 묻는 질문의 경우 회사의 주요 내용(사업,업무,제도,최근 이슈) 등을 기술하여 작성했는지 판단하여 피드백 해줘.

생각하게 된 이유, 사건, 경험에 대해서 묻는 질문은 꼭 관련 경험이 함께 작성되어있는지 판단하여 피드백 해줘.

아래 내용을 중심으로 평가해주세요:
1. 이 답변이 해당 직무 및 인재상에 적합한지
2. 보완해야 할 점이 있다면 구체적으로
3. 가산점을 줄 수 있는 요소나 표현 제안

친절하고 구체적으로, 면접관 또는 커리어 코치의 시선으로 피드백을 작성해주세요.
""".strip()

    try:
        # 2️⃣ 메시지 추가
        await openai.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_message,
        )

        # 3️⃣ Run 생성·폴링
        run = await openai.beta.threads.runs.create_and_poll(
            thread_id=thread_id,
            assistant_id=assistant_id,
        )

        # 4️⃣ 응답 추출
        messages = await openai.beta.threads.messages.list(
            thread_id=thread_id,
            run_id=run.id,
        )
        return messages.data[0].content[0].text.value.strip()

    except OpenAIError as exc:
        raise RuntimeError(f"Assistant 실행 중 오류: {exc}") from exc

# =====================================================
# 6️⃣  CLI: RAGAS 평가용 실행 (assistant_service.py 직접 실행 시)
# =====================================================
if __name__ == "__main__":
    import asyncio
    from pathlib import Path
    import json

    # ① assistant_id 로드 (.assistant.id 파일에 저장돼 있다고 가정)
    assistant_id_path = Path(".assistant.id")
    if not assistant_id_path.exists():
        raise FileNotFoundError("⚠️ .assistant.id 파일을 찾을 수 없습니다!")
    raw_id= assistant_id_path.read_text(encoding="utf-8").strip()
    assistant_id = raw_id.split("=", 1)[-1]

    # ② request_data 템플릿 (직무 JSON + question/answer 빈 칸)
    job_json_str = """
[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
    
    job_list = json.loads(job_json_str)        # 리스트 형태
    first_job = job_list[0]                    # 여기서는 첫 번째 직무만 사용
    # ③ request_data 딕셔너리 구성
    request_data: dict = {
        "company": "예시회사",                      # 필요 시 수정
        "position": first_job["직무명"],
        "qualifications": "\n".join(first_job["자격요건"]),
        "requirements":   "\n".join(first_job["필수사항"]),
        "duties":         "\n".join(first_job["담당업무"]),
        "preferred":      "\n".join(first_job["우대사항"]),
        "ideal":          "\n".join(first_job["인재상"]),
        # -------------- 질문 / 답변 (빈 값) --------------
        "question": "",
        "answer":   "",
    }

    # ④ 사용자에게 질문/답변 직접 입력받기 (원할 경우)
    print("📝 지원자 질문/답변을 입력하세요. (그냥 Enter 치면 빈 값 유지)")
    q_in = input("질문: ").strip()
    a_in = input("답변: ").strip()
    if q_in:
        request_data["question"] = q_in
    if a_in:
        request_data["answer"] = a_in

    # ⑤ 세션키는 간단히 company 이름을 사용 (테스트용)
    session_key = request_data["company"]

    # ⑥ Assistant 실행
    reply = asyncio.run(
        run_assistant(assistant_id, request_data, session_key)
    )

    print("\n========== Assistant 응답 ==========")
    print(reply)

📝 지원자 질문/답변을 입력하세요. (그냥 Enter 치면 빈 값 유지)


질문:  본인의 지식 및 기술역량을 지원직무 수행 시 어떻게 발휘할 수 있을지 설명해 주세요.
답변:  저는 요가 강사로 활동하며 사람들에게 올바른 자세와 균형을 알려주는 일을 해왔습니다. 몸과 마음의 유연성을 기르는 과정에서 꾸준함과 집중력을 배웠습니다. 이러한 경험을 살려, 귀사의 자동차 생산 공장에서 정밀한 검사 설비를 다루는 데에 차분하고 성실하게 임하겠습니다. 기계와는 아직 익숙하지 않지만, 새로운 도전을 두려워하지 않습니다.


C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:88: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  await openai.beta.threads.messages.create(
C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:95: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = await openai.beta.threads.runs.create_and_poll(
C:\Users\sdg15\AppData\Local\Temp\ipykernel_15528\2333748281.py:101: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = await openai.beta.threads.messages.list(



========== Assistant 응답 ==========
1. 적합성 평가
   - 지원자의 답변은 요가 강사로서의 경험에 집중하고 있습니다. 하지만 Tech PM 포지션은 C++, C# 언어 사용 능력과 관련된 프로그램 개발 경험이 필수적이기 때문에 지원자의 답변이 해당 직무와 전혀 맞지 않습니다. 자동차 검사 설비나 공정에 대한 이해가 부족하다는 점도 큰 단점입니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'을 나타내기 위해 요가 강사로서의 경험을 관련짓는 것도 불충분해 보입니다.

2. 보완/개선 제안
   - 지원자가 해당 직무에 적합한 지원자로 평가되기 위해서는 프로그래밍 언어와 관련된 경험을 강조해야 합니다. 요가 강사로서의 경험은 긍정적인 요소일 수 있지만, 기술적인 배경 부족을 보완할 수 없기 때문에 부족한 부분을 메울 수 있는 구체적인 사례를 추가해야 합니다. 예를 들어, "최근에 C++과 C# 언어를 학습하고 있는 중이며, 이를 통해 자동차 검사 설비와 관련된 프로젝트에 기여할 수 있는 기반을 마련하고 있습니다." 와 같은 내용이 필요합니다. 또한 자동차 검사와 관련된 지식을 쌓기 위해 학습 중인 자료나 프로젝트를 언급하면 신뢰성을 높일 수 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 강력한 인상을 남길 수 있습니다. "요가 강사로서의 경험을 통해 사람들과 소통하며 효율적인 피드백을 주고받는 기술을 익혔고, 이러한 소통 능력은 고객과의 현장 대응에서도 큰 도움이 될 것입니다."라는 표현은 소통과 협력 능력을 강조하면서 독창적인 접근을 보여줄 수 있습니다. 기술과 관련 없는 경력을 좀 더 효과적으로 활용하기 위해, "다양한 사람들과의 상호작용을 통해 팀워크를 구축하는 데 필요한 역량을 개발했습니다. 이는 생산라인 내 문제 해결에도 기여할 수 있습니다."라고 말함으로써 지원자가 가진 태도를 긍정적으로 보여줄 수 있습니다. 이러한 방식으로 답변을 구체화하고 강화시키면 면접관에게 더 긍정적

In [48]:
"""
ragas_demo.py
─────────────
Assistant 응답·질문·근거(contexts)를 수동으로 입력해
RAGAS 평가 지표(faithfulness, answer_relevancy)를 빠르게 확인하는 스크립트
"""

from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate
from datasets import Dataset  # 👈 추가
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts
    
# =====================================================
# 📝 ① 여기서 직접 입력하세요
# =====================================================
job_json_str = """
[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
job_list = json.loads(job_json_str)
# (1) contexts 생성
contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)

assistant_answer = """
1. 적합성 평가
   - 지원자의 답변은 Python 기반 검사 시스템 개발 경험을 기술하고 있으며, 이는 기술적 역량을 보여주려는 시도로 볼 수 있지만, 해당 직무인 Tech PM 포지션은 C++ 및 C#을 사용하는 프로그램 개발이 주 요구사항이므로 적합성이 부족합니다. 자동차 검사 설비와 관련된 경험을 통해 지원자가 이 직무에 어떤 기여를 할 수 있는지를 명확하게 제시할 필요가 있습니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 관련 경험이 필요하며, 이를 통해 직무와의 연관성을 높이는 것이 중요합니다.

2. 보완/개선 제안
   - 지원자는 C++ 또는 C#에 대한 경험이 부족하다면 관련 언어의 학습이나 적응 의지를 강조해야 합니다. 예를 들어, “비록 주로 Python을 사용했지만, C++ 및 C#의 기본기를 학습하고 있으며, 이를 통해 검사 설비 소프트웨어 개발에 기여하고자 합니다.”라는 내용이 좋습니다. 또한, 자동차 통신 프로토콜이나 검사 설비에 대한 이해도를 높이기 위해 관련 지식을 습득한 경험이나 내용을 포함시키는 것이 필요합니다. 현장 대응에 대한 경험이 있다면 고객과의 소통 또는 문제 해결에 대한 사례도 덧붙여, 직무에 대한 적합성을 한층 강화할 수 있습니다.

3. 가산점 요소·표현
   - 지원자가 자신의 경험을 직무에 의해 요구되는 역량과 연결해야 긍정적인 인상을 줄 수 있습니다. 예를 들어, "과거 프로젝트에서 각각의 로직을 모듈화하여 유지보수를 용이하게 만들었습니다. 이러한 시스템적 사고는 검사 설비의 S/W 개발 과정에서도 큰 도움이 될 것입니다."라는 표현은 지원자의 경험을 더 잘 직무와 연결시키는 데 도움이 됩니다. 추가로, “고객의 요구 사항을 반영하여 시스템을 개선한 경험이 있으며, 이를 통해 고객과의 관계를 더욱 돈독히 하며 효과적으로 대응할 수 있습니다.”와 같은 문장은 소통과 협력 능력을 강조할 수 있습니다. 이러한 요소들을 포함시킴으로써 지원자의 전체적인 적합성을 높일 수 있습니다.
"""

user_question = """
저는 자동차 생산 공정의 품질 확보를 위해 Python 기반 검사 시스템을 개발한 경험이 있습니다. 공정 내 부품 이상을 실시간으로 감지하는 비전 검사 프로그램을 구현하며, 카메라와 PLC 장비를 연동해 데이터를 수집하고, 이상을 탐지하면 즉시 알람을 출력하는 로직을 설계했습니다. 또한 사용자 친화적인 UI를 개발해 작업자의 효율성을 높였고, 유지보수가 쉽도록 모듈화하였습니다. 이러한 경험을 바탕으로 귀사에서 검사 설비의 소프트웨어 개발과 현장 대응에 기여하겠습니다.
"""

# =====================================================
# ② RAGAS 입력 샘플 구성
# =====================================================
samples = [
    {
        "question": user_question.strip(),
        "answer":   assistant_answer.strip(),
        "contexts": [c.strip() for c in contexts],
    }
]
dataset = Dataset.from_list(samples)
# =====================================================
# ③ RAGAS 평가 실행
# =====================================================

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy]
)

print("\n🟢 RAGAS 평가 결과")
print(result)

Evaluating: 100%|██████████| 2/2 [00:26<00:00, 13.34s/it]



🟢 RAGAS 평가 결과
{'faithfulness': 0.1111, 'answer_relevancy': 0.0000}


In [49]:
"""
ragas_demo.py
─────────────
Assistant 응답·질문·근거(contexts)를 수동으로 입력해
RAGAS 평가 지표(faithfulness, answer_relevancy)를 빠르게 확인하는 스크립트
"""

from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate
from datasets import Dataset  # 👈 추가
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts
    
# =====================================================
# 📝 ① 여기서 직접 입력하세요
# =====================================================
job_json_str = """
[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
job_list = json.loads(job_json_str)
# (1) contexts 생성
contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)

assistant_answer = """
1. 적합성 평가
   - 지원자의 답변은 주로 웹 프론트엔드 개발 경험을 강조하고 있으며, 자동차 검사 설비와의 직접적인 관련성은 부족합니다. 해당 직무가 요구하는 C++, C# 사용 경험 및 검사 설비의 소프트웨어 개발 경험이 결여되어 있어, 직무에 대한 적합성이 현저히 떨어집니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 지원자의 관련 경험이 부족하기 때문에 이를 통해 지원자가 해당 기업 및 직무에 얼마나 잘 적응할 수 있는지를 보여주는 데 한계가 있습니다.

2. 보완/개선 제안
   - 지원자는 자신의 프론트엔드 경험을 활용해 C++ 또는 C# 관련 경험을 보완하여 명확히 제시해야 합니다. 예를 들어, “비록 제 주 된 경력이 프론트엔드 개발에 국한되지만, 기본적인 C++ 및 C#를 독학하며 간단한 프로젝트에 참여했습니다.”와 같은 내용을 추가하면 좋습니다. 자동차 검사 설비에 대한 이해를 높이기 위해 관련 커리큘럼이나 개인 프로젝트 경험 등을 언급하여 역량 향상 가능성을 보여주는 것도 필요합니다. 관여해 본 고객 지원 경험이나 팀 프로젝트 내 소통 역할에 대한 사례를 포함해 소통 및 협력 능력을 강조할 필요가 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 가산점을 얻을 수 있습니다. 예를 들어, “저는 사용자 친화적인 UI 설계를 통해 다양한 사용자의 요구를 반영한 경험이 있으며, 이러한 접근 방식을 통해 검사 설비의 운영과 유지보수 과정에서도 고객의 목소리를 반영하는 데 기여할 것입니다.”라는 표현은 비단 기술적인 역량을 넘어서 지원자의 협력적 태도와 열정을 보여줄 수 있습니다. 또한, “신기술에 대한 빠른 습득을 통해 현업에서의 문제를 빠르게 해결할 수 있는 능력을 갖추고 있으며, 이는 귀사의 검사 설비 운영에 큰 도움이 될 것입니다.”와 같은 표현을 추가하여 지원자의 학습과 성장에 대한 열망을 어필하는 것이 좋습니다. 이렇게 구체적인 예시와 연결고리를 만들어 지원자가 가진 역량을 더욱 강조하기 위해 노력하는 것이 중요합니다.
"""

user_question = """
1. 적합성 평가
   - 지원자의 답변은 주로 웹 프론트엔드 개발 경험을 강조하고 있으며, 자동차 검사 설비와의 직접적인 관련성은 부족합니다. 해당 직무가 요구하는 C++, C# 사용 경험 및 검사 설비의 소프트웨어 개발 경험이 결여되어 있어, 직무에 대한 적합성이 현저히 떨어집니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 지원자의 관련 경험이 부족하기 때문에 이를 통해 지원자가 해당 기업 및 직무에 얼마나 잘 적응할 수 있는지를 보여주는 데 한계가 있습니다.

2. 보완/개선 제안
   - 지원자는 자신의 프론트엔드 경험을 활용해 C++ 또는 C# 관련 경험을 보완하여 명확히 제시해야 합니다. 예를 들어, “비록 제 주 된 경력이 프론트엔드 개발에 국한되지만, 기본적인 C++ 및 C#를 독학하며 간단한 프로젝트에 참여했습니다.”와 같은 내용을 추가하면 좋습니다. 자동차 검사 설비에 대한 이해를 높이기 위해 관련 커리큘럼이나 개인 프로젝트 경험 등을 언급하여 역량 향상 가능성을 보여주는 것도 필요합니다. 관여해 본 고객 지원 경험이나 팀 프로젝트 내 소통 역할에 대한 사례를 포함해 소통 및 협력 능력을 강조할 필요가 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 가산점을 얻을 수 있습니다. 예를 들어, “저는 사용자 친화적인 UI 설계를 통해 다양한 사용자의 요구를 반영한 경험이 있으며, 이러한 접근 방식을 통해 검사 설비의 운영과 유지보수 과정에서도 고객의 목소리를 반영하는 데 기여할 것입니다.”라는 표현은 비단 기술적인 역량을 넘어서 지원자의 협력적 태도와 열정을 보여줄 수 있습니다. 또한, “신기술에 대한 빠른 습득을 통해 현업에서의 문제를 빠르게 해결할 수 있는 능력을 갖추고 있으며, 이는 귀사의 검사 설비 운영에 큰 도움이 될 것입니다.”와 같은 표현을 추가하여 지원자의 학습과 성장에 대한 열망을 어필하는 것이 좋습니다. 이렇게 구체적인 예시와 연결고리를 만들어 지원자가 가진 역량을 더욱 강조하기 위해 노력하는 것이 중요합니다.
"""

# =====================================================
# ② RAGAS 입력 샘플 구성
# =====================================================
samples = [
    {
        "question": user_question.strip(),
        "answer":   assistant_answer.strip(),
        "contexts": [c.strip() for c in contexts],
    }
]
dataset = Dataset.from_list(samples)
# =====================================================
# ③ RAGAS 평가 실행
# =====================================================

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy]
)

print("\n🟢 RAGAS 평가 결과")
print(result)

Evaluating: 100%|██████████| 2/2 [00:27<00:00, 13.84s/it]



🟢 RAGAS 평가 결과
{'faithfulness': 0.0000, 'answer_relevancy': 0.0000}


In [50]:
"""
ragas_demo.py
─────────────
Assistant 응답·질문·근거(contexts)를 수동으로 입력해
RAGAS 평가 지표(faithfulness, answer_relevancy)를 빠르게 확인하는 스크립트
"""

from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate
from datasets import Dataset  # 👈 추가
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts
    
# =====================================================
# 📝 ① 여기서 직접 입력하세요
# =====================================================
job_json_str = """
[
  {
    "직무명": "Tech PM(Project Manager)",
    "담당업무": [
      "현대 / 기아 자동차 공장 내 생산 차량에 대한 검사 설비 개발",
      "검사 설비 개발 사양 분석",
      "검사 설비 S/W 개발 (UI 및 제어 프로그램 개발)",
      "검차 설비 구축 및 외부 장비와의 인터페이스 구성",
      "검차 설비 운영 및 유지보수",
      "생산 라인 내 설비 이슈 진단 및 문제 해결",
      "고객(공장 측)과의 현장 대응 및 기술 지원"
    ],
    "자격요건": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "필수사항": [
      "C++, C# 등 프로그래밍 언어 사용 가능자",
      "관련 프로그램 개발 경험 보유자 (경력 무관, 실무 중심이면 가능)"
    ],
    "우대사항": [
      "컴퓨터, 전자, 제어, 로봇, 소프트웨어공학 등 관련 전공자",
      "자동차 검차 설비에 대한 이해 보유자 (※ 경험이 없어도 실무를 통해 역량 향상 가능)",
      "자동차 통신 프로토콜(CAN, KWP 등) 이해",
      "진단기 및 진단 장비 활용 경험"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
job_list = json.loads(job_json_str)
# (1) contexts 생성
contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)

assistant_answer = """
1. 적합성 평가
   - 지원자의 답변은 요가 강사로서의 경험에 집중하고 있습니다. 하지만 Tech PM 포지션은 C++, C# 언어 사용 능력과 관련된 프로그램 개발 경험이 필수적이기 때문에 지원자의 답변이 해당 직무와 전혀 맞지 않습니다. 자동차 검사 설비나 공정에 대한 이해가 부족하다는 점도 큰 단점입니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'을 나타내기 위해 요가 강사로서의 경험을 관련짓는 것도 불충분해 보입니다.

2. 보완/개선 제안
   - 지원자가 해당 직무에 적합한 지원자로 평가되기 위해서는 프로그래밍 언어와 관련된 경험을 강조해야 합니다. 요가 강사로서의 경험은 긍정적인 요소일 수 있지만, 기술적인 배경 부족을 보완할 수 없기 때문에 부족한 부분을 메울 수 있는 구체적인 사례를 추가해야 합니다. 예를 들어, "최근에 C++과 C# 언어를 학습하고 있는 중이며, 이를 통해 자동차 검사 설비와 관련된 프로젝트에 기여할 수 있는 기반을 마련하고 있습니다." 와 같은 내용이 필요합니다. 또한 자동차 검사와 관련된 지식을 쌓기 위해 학습 중인 자료나 프로젝트를 언급하면 신뢰성을 높일 수 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 강력한 인상을 남길 수 있습니다. "요가 강사로서의 경험을 통해 사람들과 소통하며 효율적인 피드백을 주고받는 기술을 익혔고, 이러한 소통 능력은 고객과의 현장 대응에서도 큰 도움이 될 것입니다."라는 표현은 소통과 협력 능력을 강조하면서 독창적인 접근을 보여줄 수 있습니다. 기술과 관련 없는 경력을 좀 더 효과적으로 활용하기 위해, "다양한 사람들과의 상호작용을 통해 팀워크를 구축하는 데 필요한 역량을 개발했습니다. 이는 생산라인 내 문제 해결에도 기여할 수 있습니다."라고 말함으로써 지원자가 가진 태도를 긍정적으로 보여줄 수 있습니다. 이러한 방식으로 답변을 구체화하고 강화시키면 면접관에게 더 긍정적인 인상을 줄 수 있습니다.
"""

user_question = """
저는 요가 강사로 활동하며 사람들에게 올바른 자세와 균형을 알려주는 일을 해왔습니다. 몸과 마음의 유연성을 기르는 과정에서 꾸준함과 집중력을 배웠습니다. 이러한 경험을 살려, 귀사의 자동차 생산 공장에서 정밀한 검사 설비를 다루는 데에 차분하고 성실하게 임하겠습니다. 기계와는 아직 익숙하지 않지만, 새로운 도전을 두려워하지 않습니다.
"""

# =====================================================
# ② RAGAS 입력 샘플 구성
# =====================================================
samples = [
    {
        "question": user_question.strip(),
        "answer":   assistant_answer.strip(),
        "contexts": [c.strip() for c in contexts],
    }
]
dataset = Dataset.from_list(samples)
# =====================================================
# ③ RAGAS 평가 실행
# =====================================================

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy]
)

print("\n🟢 RAGAS 평가 결과")
print(result)

Evaluating: 100%|██████████| 2/2 [00:37<00:00, 18.51s/it]



🟢 RAGAS 평가 결과
{'faithfulness': 0.1053, 'answer_relevancy': 0.0000}


In [51]:
"""
ragas_demo.py
─────────────
Assistant 응답·질문·근거(contexts)를 수동으로 입력해
RAGAS 평가 지표(faithfulness, answer_relevancy)를 빠르게 확인하는 스크립트
"""

from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate
from datasets import Dataset  # 👈 추가
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts
    
# =====================================================
# 📝 ① 여기서 직접 입력하세요
# =====================================================
# job_json_str = """
# [
#   {
#     "직무명": "연구소 서버개발 파트",
#     "담당업무": [
#       "현대기아 진단기 연계 Web 플랫폼 개발- eReport 전세계 차량 진단 결과를 수집/조회/통계/타 시스템 연동을 위한 서비스 개발: IT프로젝트 수주 시 개발 착수, 요건 분석, 설계, IF 부분 연동, 백엔드/프론트엔드 설계/기획 및 백엔드/서버단 서비스 개발",
#       "디지털 필드픽스 및 진단기운영/인증, 데이터 수집 연동, 캠페인, ECU업그레이드Web 부분 서비스 개발/대응",
#       "본사-지사(북미/유럽/중국) 간 진단기 서비스 및 신규 개발 안건 검토 및 수행",
#       "현대/기아 공식 진단 정비 Web 서비스 개발- 진단서비스 : .Net 및 Java 기반 공식 진단 정비 웹 서비스 개발",
#       "PDI 서비스 : PDI 검사 결과 웹 서비스 개발 (북미/유럽/중국/내수) 및 사이트 운영",
#       "고객사와 요건 분석/협의- 고객의 신규 기능 개발 요구사항 분석 : 고객 요구사항 분석, 기술검토, 일정 검토를 기반으로 프로젝트 사전검토 및 고객협의"
#     ],
#     "자격요건": [
#       "경력 5년 이상"
#     ],
#     "필수사항": [
#       ".Net/Java 개발 중급 이상 (DB - MSSQL / TIBERO / ORACLE 유경험자)",
#       "서버운영 및 웹서비스에 대한 이해",
#       "코드 리뷰 경험 다수"
#     ],
#     "우대사항": [
#       "웹 서비스(.Net/Java-spring,spring boot) 개발을 5년 이상 수행하며, 서비스 운영해 본 분",
#       "백엔드/프론트 엔드, Third 서비스 연동 개발 가능한 분",
#       "고객 요구사항에 대한 분석/검토 가능한 분",
#       "프론트엔드, 백엔드 개발언어들을 다룰 수 있는 분",
#       "데이타베이스(MS-SQL,TIBERO,ORACLE)에 대한 전반적인 지식 및 튜닝작업에 대해 이해가 있는 분",
#       "서버(Windows)에 대한 관리 및 운영을 해보신 분"
#     ],
#     "인재상": [
#       "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
#     ]
#   }
# ]
# """.strip()
# job_list = json.loads(job_json_str)
# # (1) contexts 생성
# contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)

contexts = [
    (
       "Tech PM(Project Manager)은 현대/기아 자동차 공장 내 생산 차량 검사 설비를 기획·개발·운영하는 역할입니다. "
        "주요 업무로는 검사 설비 사양 분석, C++·C# 기반 UI·제어 프로그램 개발, 검차 설비 구축 및 외부 장비 인터페이스 구성, "
        "생산라인 설비 이슈 진단·문제 해결, 그리고 공장 측 고객과의 현장 대응·기술 지원이 포함됩니다. "
        "필수 요건은 C++·C# 등 프로그래밍 언어 사용 능력과 관련 프로그램 개발 경험이며, "
        "우대 사항으로는 컴퓨터·전자·제어·로봇 전공, 자동차 검차 설비 이해, 자동차 통신 프로토콜(CAN·KWP) 이해, "
        "진단기·진단 장비 활용 경험이 제시됩니다. "
        "인재상은 열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장을 강조합니다."
    )
]

assistant_answer = """
1. 적합성 평가
   - 지원자의 답변은 Python 기반 검사 시스템 개발 경험을 기술하고 있으며, 이는 기술적 역량을 보여주려는 시도로 볼 수 있지만, 해당 직무인 Tech PM 포지션은 C++ 및 C#을 사용하는 프로그램 개발이 주 요구사항이므로 적합성이 부족합니다. 자동차 검사 설비와 관련된 경험을 통해 지원자가 이 직무에 어떤 기여를 할 수 있는지를 명확하게 제시할 필요가 있습니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 관련 경험이 필요하며, 이를 통해 직무와의 연관성을 높이는 것이 중요합니다.

2. 보완/개선 제안
   - 지원자는 C++ 또는 C#에 대한 경험이 부족하다면 관련 언어의 학습이나 적응 의지를 강조해야 합니다. 예를 들어, “비록 주로 Python을 사용했지만, C++ 및 C#의 기본기를 학습하고 있으며, 이를 통해 검사 설비 소프트웨어 개발에 기여하고자 합니다.”라는 내용이 좋습니다. 또한, 자동차 통신 프로토콜이나 검사 설비에 대한 이해도를 높이기 위해 관련 지식을 습득한 경험이나 내용을 포함시키는 것이 필요합니다. 현장 대응에 대한 경험이 있다면 고객과의 소통 또는 문제 해결에 대한 사례도 덧붙여, 직무에 대한 적합성을 한층 강화할 수 있습니다.

3. 가산점 요소·표현
   - 지원자가 자신의 경험을 직무에 의해 요구되는 역량과 연결해야 긍정적인 인상을 줄 수 있습니다. 예를 들어, "과거 프로젝트에서 각각의 로직을 모듈화하여 유지보수를 용이하게 만들었습니다. 이러한 시스템적 사고는 검사 설비의 S/W 개발 과정에서도 큰 도움이 될 것입니다."라는 표현은 지원자의 경험을 더 잘 직무와 연결시키는 데 도움이 됩니다. 추가로, “고객의 요구 사항을 반영하여 시스템을 개선한 경험이 있으며, 이를 통해 고객과의 관계를 더욱 돈독히 하며 효과적으로 대응할 수 있습니다.”와 같은 문장은 소통과 협력 능력을 강조할 수 있습니다. 이러한 요소들을 포함시킴으로써 지원자의 전체적인 적합성을 높일 수 있습니다.
"""

user_question = """
저는 자동차 생산 공정의 품질 확보를 위해 Python 기반 검사 시스템을 개발한 경험이 있습니다. 공정 내 부품 이상을 실시간으로 감지하는 비전 검사 프로그램을 구현하며, 카메라와 PLC 장비를 연동해 데이터를 수집하고, 이상을 탐지하면 즉시 알람을 출력하는 로직을 설계했습니다. 또한 사용자 친화적인 UI를 개발해 작업자의 효율성을 높였고, 유지보수가 쉽도록 모듈화하였습니다. 이러한 경험을 바탕으로 귀사에서 검사 설비의 소프트웨어 개발과 현장 대응에 기여하겠습니다.
"""

# =====================================================
# ② RAGAS 입력 샘플 구성
# =====================================================
samples = [
    {
        "question": user_question.strip(),
        "answer":   assistant_answer.strip(),
        "contexts": [c.strip() for c in contexts],
    }
]
dataset = Dataset.from_list(samples)
# =====================================================
# ③ RAGAS 평가 실행
# =====================================================

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy]
)

print("\n🟢 RAGAS 평가 결과")
print(result)

Evaluating: 100%|██████████| 2/2 [00:19<00:00,  9.80s/it]



🟢 RAGAS 평가 결과
{'faithfulness': 0.7222, 'answer_relevancy': 0.0000}


In [52]:
"""
ragas_demo.py
─────────────
Assistant 응답·질문·근거(contexts)를 수동으로 입력해
RAGAS 평가 지표(faithfulness, answer_relevancy)를 빠르게 확인하는 스크립트
"""

from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate
from datasets import Dataset  # 👈 추가
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts
    
# =====================================================
# 📝 ① 여기서 직접 입력하세요
# =====================================================
# job_json_str = """
# [
#   {
#     "직무명": "연구소 서버개발 파트",
#     "담당업무": [
#       "현대기아 진단기 연계 Web 플랫폼 개발- eReport 전세계 차량 진단 결과를 수집/조회/통계/타 시스템 연동을 위한 서비스 개발: IT프로젝트 수주 시 개발 착수, 요건 분석, 설계, IF 부분 연동, 백엔드/프론트엔드 설계/기획 및 백엔드/서버단 서비스 개발",
#       "디지털 필드픽스 및 진단기운영/인증, 데이터 수집 연동, 캠페인, ECU업그레이드Web 부분 서비스 개발/대응",
#       "본사-지사(북미/유럽/중국) 간 진단기 서비스 및 신규 개발 안건 검토 및 수행",
#       "현대/기아 공식 진단 정비 Web 서비스 개발- 진단서비스 : .Net 및 Java 기반 공식 진단 정비 웹 서비스 개발",
#       "PDI 서비스 : PDI 검사 결과 웹 서비스 개발 (북미/유럽/중국/내수) 및 사이트 운영",
#       "고객사와 요건 분석/협의- 고객의 신규 기능 개발 요구사항 분석 : 고객 요구사항 분석, 기술검토, 일정 검토를 기반으로 프로젝트 사전검토 및 고객협의"
#     ],
#     "자격요건": [
#       "경력 5년 이상"
#     ],
#     "필수사항": [
#       ".Net/Java 개발 중급 이상 (DB - MSSQL / TIBERO / ORACLE 유경험자)",
#       "서버운영 및 웹서비스에 대한 이해",
#       "코드 리뷰 경험 다수"
#     ],
#     "우대사항": [
#       "웹 서비스(.Net/Java-spring,spring boot) 개발을 5년 이상 수행하며, 서비스 운영해 본 분",
#       "백엔드/프론트 엔드, Third 서비스 연동 개발 가능한 분",
#       "고객 요구사항에 대한 분석/검토 가능한 분",
#       "프론트엔드, 백엔드 개발언어들을 다룰 수 있는 분",
#       "데이타베이스(MS-SQL,TIBERO,ORACLE)에 대한 전반적인 지식 및 튜닝작업에 대해 이해가 있는 분",
#       "서버(Windows)에 대한 관리 및 운영을 해보신 분"
#     ],
#     "인재상": [
#       "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
#     ]
#   }
# ]
# """.strip()
# job_list = json.loads(job_json_str)
# # (1) contexts 생성
# contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)

contexts = [
    (
        "Tech PM(Project Manager)은 현대/기아 자동차 공장 내 생산 차량 검사 설비를 기획·개발·운영하는 역할입니다. "
        "주요 업무로는 검사 설비 사양 분석, C++·C# 기반 UI·제어 프로그램 개발, 검차 설비 구축 및 외부 장비 인터페이스 구성, "
        "생산라인 설비 이슈 진단·문제 해결, 그리고 공장 측 고객과의 현장 대응·기술 지원이 포함됩니다. "
        "필수 요건은 C++·C# 등 프로그래밍 언어 사용 능력과 관련 프로그램 개발 경험이며, "
        "우대 사항으로는 컴퓨터·전자·제어·로봇 전공, 자동차 검차 설비 이해, 자동차 통신 프로토콜(CAN·KWP) 이해, "
        "진단기·진단 장비 활용 경험이 제시됩니다. "
        "인재상은 열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장을 강조합니다."
    )
]

assistant_answer = """
1. 적합성 평가
   - 지원자의 답변은 주로 웹 프론트엔드 개발 경험을 강조하고 있으며, 자동차 검사 설비와의 직접적인 관련성은 부족합니다. 해당 직무가 요구하는 C++, C# 사용 경험 및 검사 설비의 소프트웨어 개발 경험이 결여되어 있어, 직무에 대한 적합성이 현저히 떨어집니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'과 같은 가치관에 대한 언급이나 지원자의 관련 경험이 부족하기 때문에 이를 통해 지원자가 해당 기업 및 직무에 얼마나 잘 적응할 수 있는지를 보여주는 데 한계가 있습니다.

2. 보완/개선 제안
   - 지원자는 자신의 프론트엔드 경험을 활용해 C++ 또는 C# 관련 경험을 보완하여 명확히 제시해야 합니다. 예를 들어, “비록 제 주 된 경력이 프론트엔드 개발에 국한되지만, 기본적인 C++ 및 C#를 독학하며 간단한 프로젝트에 참여했습니다.”와 같은 내용을 추가하면 좋습니다. 자동차 검사 설비에 대한 이해를 높이기 위해 관련 커리큘럼이나 개인 프로젝트 경험 등을 언급하여 역량 향상 가능성을 보여주는 것도 필요합니다. 관여해 본 고객 지원 경험이나 팀 프로젝트 내 소통 역할에 대한 사례를 포함해 소통 및 협력 능력을 강조할 필요가 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 가산점을 얻을 수 있습니다. 예를 들어, “저는 사용자 친화적인 UI 설계를 통해 다양한 사용자의 요구를 반영한 경험이 있으며, 이러한 접근 방식을 통해 검사 설비의 운영과 유지보수 과정에서도 고객의 목소리를 반영하는 데 기여할 것입니다.”라는 표현은 비단 기술적인 역량을 넘어서 지원자의 협력적 태도와 열정을 보여줄 수 있습니다. 또한, “신기술에 대한 빠른 습득을 통해 현업에서의 문제를 빠르게 해결할 수 있는 능력을 갖추고 있으며, 이는 귀사의 검사 설비 운영에 큰 도움이 될 것입니다.”와 같은 표현을 추가하여 지원자의 학습과 성장에 대한 열망을 어필하는 것이 좋습니다. 이렇게 구체적인 예시와 연결고리를 만들어 지원자가 가진 역량을 더욱 강조하기 위해 노력하는 것이 중요합니다.
"""

user_question = """
저는 주로 웹 프론트엔드 개발을 수행해왔고, 자동차 검사 설비 개발과는 직접적인 경험이 없습니다. 하지만 다양한 서비스에서 사용자 친화적 UI와 유지보수가 용이한 코드 구조를 설계해온 경험이 있으며, 현장의 데이터를 시각화하거나 관리자가 쉽게 사용할 수 있는 인터페이스를 구축하는 데에는 자신 있습니다. 새로운 환경과 기술을 빠르게 습득하는 강점으로 귀사의 검사 설비 운영과 유지보수에 기여하겠습니다.
"""

# =====================================================
# ② RAGAS 입력 샘플 구성
# =====================================================
samples = [
    {
        "question": user_question.strip(),
        "answer":   assistant_answer.strip(),
        "contexts": [c.strip() for c in contexts],
    }
]
dataset = Dataset.from_list(samples)
# =====================================================
# ③ RAGAS 평가 실행
# =====================================================

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy]
)

print("\n🟢 RAGAS 평가 결과")
print(result)

Evaluating: 100%|██████████| 2/2 [00:23<00:00, 11.55s/it]



🟢 RAGAS 평가 결과
{'faithfulness': 0.0000, 'answer_relevancy': 0.0000}


In [53]:
"""
ragas_demo.py
─────────────
Assistant 응답·질문·근거(contexts)를 수동으로 입력해
RAGAS 평가 지표(faithfulness, answer_relevancy)를 빠르게 확인하는 스크립트
"""

from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate
from datasets import Dataset  # 👈 추가
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts
    
# =====================================================
# 📝 ① 여기서 직접 입력하세요
# =====================================================
# job_json_str = """
# [
#   {
#     "직무명": "연구소 서버개발 파트",
#     "담당업무": [
#       "현대기아 진단기 연계 Web 플랫폼 개발- eReport 전세계 차량 진단 결과를 수집/조회/통계/타 시스템 연동을 위한 서비스 개발: IT프로젝트 수주 시 개발 착수, 요건 분석, 설계, IF 부분 연동, 백엔드/프론트엔드 설계/기획 및 백엔드/서버단 서비스 개발",
#       "디지털 필드픽스 및 진단기운영/인증, 데이터 수집 연동, 캠페인, ECU업그레이드Web 부분 서비스 개발/대응",
#       "본사-지사(북미/유럽/중국) 간 진단기 서비스 및 신규 개발 안건 검토 및 수행",
#       "현대/기아 공식 진단 정비 Web 서비스 개발- 진단서비스 : .Net 및 Java 기반 공식 진단 정비 웹 서비스 개발",
#       "PDI 서비스 : PDI 검사 결과 웹 서비스 개발 (북미/유럽/중국/내수) 및 사이트 운영",
#       "고객사와 요건 분석/협의- 고객의 신규 기능 개발 요구사항 분석 : 고객 요구사항 분석, 기술검토, 일정 검토를 기반으로 프로젝트 사전검토 및 고객협의"
#     ],
#     "자격요건": [
#       "경력 5년 이상"
#     ],
#     "필수사항": [
#       ".Net/Java 개발 중급 이상 (DB - MSSQL / TIBERO / ORACLE 유경험자)",
#       "서버운영 및 웹서비스에 대한 이해",
#       "코드 리뷰 경험 다수"
#     ],
#     "우대사항": [
#       "웹 서비스(.Net/Java-spring,spring boot) 개발을 5년 이상 수행하며, 서비스 운영해 본 분",
#       "백엔드/프론트 엔드, Third 서비스 연동 개발 가능한 분",
#       "고객 요구사항에 대한 분석/검토 가능한 분",
#       "프론트엔드, 백엔드 개발언어들을 다룰 수 있는 분",
#       "데이타베이스(MS-SQL,TIBERO,ORACLE)에 대한 전반적인 지식 및 튜닝작업에 대해 이해가 있는 분",
#       "서버(Windows)에 대한 관리 및 운영을 해보신 분"
#     ],
#     "인재상": [
#       "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
#     ]
#   }
# ]
# """.strip()
# job_list = json.loads(job_json_str)
# # (1) contexts 생성
# contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)

contexts = [
    (
        "Tech PM(Project Manager)은 현대/기아 자동차 공장 내 생산 차량 검사 설비를 기획·개발·운영하는 역할입니다. "
        "주요 업무로는 검사 설비 사양 분석, C++·C# 기반 UI·제어 프로그램 개발, 검차 설비 구축 및 외부 장비 인터페이스 구성, "
        "생산라인 설비 이슈 진단·문제 해결, 그리고 공장 측 고객과의 현장 대응·기술 지원이 포함됩니다. "
        "필수 요건은 C++·C# 등 프로그래밍 언어 사용 능력과 관련 프로그램 개발 경험이며, "
        "우대 사항으로는 컴퓨터·전자·제어·로봇 전공, 자동차 검차 설비 이해, 자동차 통신 프로토콜(CAN·KWP) 이해, "
        "진단기·진단 장비 활용 경험이 제시됩니다. "
        "인재상은 열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장을 강조합니다."
    )
]

assistant_answer = """
1. 적합성 평가
   - 지원자의 답변은 요가 강사로서의 경험에 집중하고 있습니다. 하지만 Tech PM 포지션은 C++, C# 언어 사용 능력과 관련된 프로그램 개발 경험이 필수적이기 때문에 지원자의 답변이 해당 직무와 전혀 맞지 않습니다. 자동차 검사 설비나 공정에 대한 이해가 부족하다는 점도 큰 단점입니다. 인재상이 요구하는 '열정과 도전, 소통과 협력'을 나타내기 위해 요가 강사로서의 경험을 관련짓는 것도 불충분해 보입니다.

2. 보완/개선 제안
   - 지원자가 해당 직무에 적합한 지원자로 평가되기 위해서는 프로그래밍 언어와 관련된 경험을 강조해야 합니다. 요가 강사로서의 경험은 긍정적인 요소일 수 있지만, 기술적인 배경 부족을 보완할 수 없기 때문에 부족한 부분을 메울 수 있는 구체적인 사례를 추가해야 합니다. 예를 들어, "최근에 C++과 C# 언어를 학습하고 있는 중이며, 이를 통해 자동차 검사 설비와 관련된 프로젝트에 기여할 수 있는 기반을 마련하고 있습니다." 와 같은 내용이 필요합니다. 또한 자동차 검사와 관련된 지식을 쌓기 위해 학습 중인 자료나 프로젝트를 언급하면 신뢰성을 높일 수 있습니다.

3. 가산점 요소·표현
   - 지원자는 자신의 경험을 직무와 연결짓는 방식으로 강력한 인상을 남길 수 있습니다. "요가 강사로서의 경험을 통해 사람들과 소통하며 효율적인 피드백을 주고받는 기술을 익혔고, 이러한 소통 능력은 고객과의 현장 대응에서도 큰 도움이 될 것입니다."라는 표현은 소통과 협력 능력을 강조하면서 독창적인 접근을 보여줄 수 있습니다. 기술과 관련 없는 경력을 좀 더 효과적으로 활용하기 위해, "다양한 사람들과의 상호작용을 통해 팀워크를 구축하는 데 필요한 역량을 개발했습니다. 이는 생산라인 내 문제 해결에도 기여할 수 있습니다."라고 말함으로써 지원자가 가진 태도를 긍정적으로 보여줄 수 있습니다. 이러한 방식으로 답변을 구체화하고 강화시키면 면접관에게 더 긍정적인 인상을 줄 수 있습니다.
"""

user_question = """
저는 요가 강사로 활동하며 사람들에게 올바른 자세와 균형을 알려주는 일을 해왔습니다. 몸과 마음의 유연성을 기르는 과정에서 꾸준함과 집중력을 배웠습니다. 이러한 경험을 살려, 귀사의 자동차 생산 공장에서 정밀한 검사 설비를 다루는 데에 차분하고 성실하게 임하겠습니다. 기계와는 아직 익숙하지 않지만, 새로운 도전을 두려워하지 않습니다.
"""

# =====================================================
# ② RAGAS 입력 샘플 구성
# =====================================================
samples = [
    {
        "question": user_question.strip(),
        "answer":   assistant_answer.strip(),
        "contexts": [c.strip() for c in contexts],
    }
]
dataset = Dataset.from_list(samples)
# =====================================================
# ③ RAGAS 평가 실행
# =====================================================

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy]
)

print("\n🟢 RAGAS 평가 결과")
print(result)

Evaluating: 100%|██████████| 2/2 [00:21<00:00, 10.50s/it]



🟢 RAGAS 평가 결과
{'faithfulness': 0.5556, 'answer_relevancy': 0.0000}


In [24]:
### 실험용
import json
from typing import List, Dict

# ─────────────────────────────────────────
# 1) job_list → contexts 리스트 변환
# ─────────────────────────────────────────
def job_list_to_contexts(job_list: List[Dict], truncate: bool = False, max_items: int = 5) -> List[str]:
    """
    job_list (파싱된 리스트[dict])를 RAGAS용 contexts 리스트로 변환
    - truncate=True 이면 각 항목을 max_items개까지만 넣고 '... (생략)' 추가
    """
    contexts: List[str] = []

    for job in job_list:
        # 직무명
        contexts.append(f"직무명: {job.get('직무명', '')}")

        # helper 내부 함수
        def add_items(prefix: str, items: List[str]):
            for i, item in enumerate(items):
                if truncate and i >= max_items:
                    contexts.append(f"{prefix}: ... (생략)")
                    break
                contexts.append(f"{prefix}: {item}")

        add_items("담당업무", job.get("담당업무", []))
        add_items("자격요건", job.get("자격요건", []))
        add_items("필수사항", job.get("필수사항", []))
        add_items("우대사항", job.get("우대사항", []))

        # 인재상은 한 줄로 합쳐도 무방
        if "인재상" in job:
            contexts.append(f"인재상: {', '.join(job['인재상'])}")

    return contexts


# ─────────────────────────────────────────
# 2) job_list[0] → request_data 딕셔너리 변환
# ─────────────────────────────────────────
def job_to_request_data(job: Dict, company: str = "지아이티") -> Dict:
    """
    한 개의 직무 dict를 Assistant용 request_data 형식으로 변환
    (question/answer 빈칸)
    """
    return {
        "company": company,
        "position": job.get("직무명", ""),
        "qualifications": "\n".join(job.get("자격요건", [])),
        "requirements":  "\n".join(job.get("필수사항", [])),
        "duties":        "\n".join(job.get("담당업무", [])),
        "preferred":     "\n".join(job.get("우대사항", [])),
        "ideal":         "\n".join(job.get("인재상", [])),
        "question": "",
        "answer":   "",
    }


# ─────────────────────────────────────────
# 3) 사용 예시
# ─────────────────────────────────────────
job_json_str = """
[
  {
    "직무명": "연구소 서버개발 파트",
    "담당업무": [
      "현대기아 진단기 연계 Web 플랫폼 개발- eReport 전세계 차량 진단 결과를 수집/조회/통계/타 시스템 연동을 위한 서비스 개발: IT프로젝트 수주 시 개발 착수, 요건 분석, 설계, IF 부분 연동, 백엔드/프론트엔드 설계/기획 및 백엔드/서버단 서비스 개발",
      "디지털 필드픽스 및 진단기운영/인증, 데이터 수집 연동, 캠페인, ECU업그레이드Web 부분 서비스 개발/대응",
      "본사-지사(북미/유럽/중국) 간 진단기 서비스 및 신규 개발 안건 검토 및 수행",
      "현대/기아 공식 진단 정비 Web 서비스 개발- 진단서비스 : .Net 및 Java 기반 공식 진단 정비 웹 서비스 개발",
      "PDI 서비스 : PDI 검사 결과 웹 서비스 개발 (북미/유럽/중국/내수) 및 사이트 운영",
      "고객사와 요건 분석/협의- 고객의 신규 기능 개발 요구사항 분석 : 고객 요구사항 분석, 기술검토, 일정 검토를 기반으로 프로젝트 사전검토 및 고객협의"
    ],
    "자격요건": [
      "경력 5년 이상"
    ],
    "필수사항": [
      ".Net/Java 개발 중급 이상 (DB - MSSQL / TIBERO / ORACLE 유경험자)",
      "서버운영 및 웹서비스에 대한 이해",
      "코드 리뷰 경험 다수"
    ],
    "우대사항": [
      "웹 서비스(.Net/Java-spring,spring boot) 개발을 5년 이상 수행하며, 서비스 운영해 본 분",
      "백엔드/프론트 엔드, Third 서비스 연동 개발 가능한 분",
      "고객 요구사항에 대한 분석/검토 가능한 분",
      "프론트엔드, 백엔드 개발언어들을 다룰 수 있는 분",
      "데이타베이스(MS-SQL,TIBERO,ORACLE)에 대한 전반적인 지식 및 튜닝작업에 대해 이해가 있는 분",
      "서버(Windows)에 대한 관리 및 운영을 해보신 분"
    ],
    "인재상": [
      "열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장"
    ]
  }
]
""".strip()
job_list = json.loads(job_json_str)

# (1) contexts 생성
contexts = job_list_to_contexts(job_list, truncate=True, max_items=3)
print("★ contexts 리스트 예시")
for c in contexts:
    print("-", c)

# (2) 첫 직무로 request_data 생성
request_data = job_to_request_data(job_list[0], company="예시회사")
print("\n★ request_data 예시")
print(json.dumps(request_data, ensure_ascii=False, indent=2))


★ contexts 리스트 예시
- 직무명: 연구소 서버개발 파트
- 담당업무: 현대기아 진단기 연계 Web 플랫폼 개발- eReport 전세계 차량 진단 결과를 수집/조회/통계/타 시스템 연동을 위한 서비스 개발: IT프로젝트 수주 시 개발 착수, 요건 분석, 설계, IF 부분 연동, 백엔드/프론트엔드 설계/기획 및 백엔드/서버단 서비스 개발
- 담당업무: 디지털 필드픽스 및 진단기운영/인증, 데이터 수집 연동, 캠페인, ECU업그레이드Web 부분 서비스 개발/대응
- 담당업무: 본사-지사(북미/유럽/중국) 간 진단기 서비스 및 신규 개발 안건 검토 및 수행
- 담당업무: ... (생략)
- 자격요건: 경력 5년 이상
- 필수사항: .Net/Java 개발 중급 이상 (DB - MSSQL / TIBERO / ORACLE 유경험자)
- 필수사항: 서버운영 및 웹서비스에 대한 이해
- 필수사항: 코드 리뷰 경험 다수
- 우대사항: 웹 서비스(.Net/Java-spring,spring boot) 개발을 5년 이상 수행하며, 서비스 운영해 본 분
- 우대사항: 백엔드/프론트 엔드, Third 서비스 연동 개발 가능한 분
- 우대사항: 고객 요구사항에 대한 분석/검토 가능한 분
- 우대사항: ... (생략)
- 인재상: 열정과 도전, 소통과 협력, 창의와 혁신, 학습과 성장

★ request_data 예시
{
  "company": "예시회사",
  "position": "연구소 서버개발 파트",
  "qualifications": "경력 5년 이상",
  "requirements": ".Net/Java 개발 중급 이상 (DB - MSSQL / TIBERO / ORACLE 유경험자)\n서버운영 및 웹서비스에 대한 이해\n코드 리뷰 경험 다수",
  "duties": "현대기아 진단기 연계 Web 플랫폼 개발- eReport 전세계 차량 진단 결과를 수집/조회/통계/타 시스템 연동을 위한 서비스 개발: IT프로젝트 수주 시 개발 착수, 요건 분석, 설계, IF 부분 연